# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

In [69]:
import pandas as pd
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [70]:
data = pd.read_csv('/Users/chrislacarte/Desktop/Labs_Ironhack/w8/2/lab-nlp/training.1600000.processed.noemoticon.csv', header=None, encoding = 'latin-1')
data.columns = ['target','id','date','flag','user','text']

In [71]:
data

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [72]:
# We take a sample of 20000 rows to make the code run faster when processing
data_s = data.sample(20000)
data_s.to_csv('data.csv')

In [73]:
sample = pd.read_csv('data.csv')

In [74]:
sample = sample.drop("Unnamed: 0", axis=1)

In [75]:
sample

,target,id,date,flag,user,text
0,0,2262449609,Sat Jun 20 22:32:03 PDT 2009,NO_QUERY,thorgalsen,"cvs, mall, stop &amp; shop, wal-mart, &amp; cl..."
1,4,2185030529,Mon Jun 15 16:46:50 PDT 2009,NO_QUERY,haliiii,Just bought so much stuff from cvs... 3D SIDEW...
2,4,2016499230,Wed Jun 03 06:53:03 PDT 2009,NO_QUERY,JRink22,4 hrs of sleep - not cool. back to bed ... MY ...
3,4,1956836172,Thu May 28 22:48:52 PDT 2009,NO_QUERY,MzCandyLand,Itz always nice having a friend who is willing...
4,0,1752440879,Sat May 09 21:55:59 PDT 2009,NO_QUERY,SammyJonas,didn't get the answer she wanted... This reall...
...,...,...,...,...,...,...
19995,4,1977985185,Sat May 30 22:09:37 PDT 2009,NO_QUERY,aka55,@CateP36 slightly more domesticated than most.
19996,0,2047804404,Fri Jun 05 14:12:22 PDT 2009,NO_QUERY,rlangdon,@SarahPrevette Oh my gosh! Sarah! Please be ok...
19997,0,2322001831,Wed Jun 24 21:53:45 PDT 2009,NO_QUERY,Jennylovesyou6,@Carale I'm currently watching Season 4 Episod...
19998,0,1691228340,Sun May 03 17:23:21 PDT 2009,NO_QUERY,Ciwwaffan10111,You know its bad when you know what episode of...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [76]:
# functions:

def clean_up(s): 
    s = s.lower()
    s = re.sub("http:\S+", " ", s)
    s = re.findall("[a-z]+", s)
    return ' '.join(s)

def tokenize(s):
    return nltk.word_tokenize(s)

def stem_and_lemmatize(l):

    ps = PorterStemmer()
    stemmed = [ps.stem(w) for w in l]

    lemmatizer = WordNetLemmatizer() 
    lemmatized = [lemmatizer.lemmatize(word) for word in l]

    return lemmatized

def remove_stopwords(l):
    return list(set(l).difference(stopwords.words()))

In [77]:
# we pass the text through the cleaning functions 
clean = []
for i in sample['text']:
    clean.append(remove_stopwords(stem_and_lemmatize(tokenize(clean_up(i)))))

In [78]:
txt_join = []
for i in clean:
    #i = " ".join(i)
    txt_join.append(i)

In [79]:
sample['text_clean'] = txt_join

In [80]:
sample['text_clean']

0        [txt, wal, amp, shop, stop, mall, mart, cleani...
1                     [chalk, bought, stuff, sidewalk, cv]
2                             [back, bed, sleep, cool, hr]
3           [lay, friend, wrote, poem, nice, itz, feeling]
4                                   [answer, wanted, suck]
                               ...                        
19995                      [slightly, domesticated, catep]
19996       [long, cent, live, sarah, sarahprevette, gosh]
19997    [season, episode, wa, killing, eli, carale, du...
19998     [house, holy, social, episode, show, shit, life]
19999                     [missbethann, metsies, boo, hoo]
Name: text_clean, Length: 20000, dtype: object

In [81]:
sample

,target,id,date,flag,user,text,text_clean
0,0,2262449609,Sat Jun 20 22:32:03 PDT 2009,NO_QUERY,thorgalsen,"cvs, mall, stop &amp; shop, wal-mart, &amp; cl...","[txt, wal, amp, shop, stop, mall, mart, cleani..."
1,4,2185030529,Mon Jun 15 16:46:50 PDT 2009,NO_QUERY,haliiii,Just bought so much stuff from cvs... 3D SIDEW...,"[chalk, bought, stuff, sidewalk, cv]"
2,4,2016499230,Wed Jun 03 06:53:03 PDT 2009,NO_QUERY,JRink22,4 hrs of sleep - not cool. back to bed ... MY ...,"[back, bed, sleep, cool, hr]"
3,4,1956836172,Thu May 28 22:48:52 PDT 2009,NO_QUERY,MzCandyLand,Itz always nice having a friend who is willing...,"[lay, friend, wrote, poem, nice, itz, feeling]"
4,0,1752440879,Sat May 09 21:55:59 PDT 2009,NO_QUERY,SammyJonas,didn't get the answer she wanted... This reall...,"[answer, wanted, suck]"
...,...,...,...,...,...,...,...
19995,4,1977985185,Sat May 30 22:09:37 PDT 2009,NO_QUERY,aka55,@CateP36 slightly more domesticated than most.,"[slightly, domesticated, catep]"
19996,0,2047804404,Fri Jun 05 14:12:22 PDT 2009,NO_QUERY,rlangdon,@SarahPrevette Oh my gosh! Sarah! Please be ok...,"[long, cent, live, sarah, sarahprevette, gosh]"
19997,0,2322001831,Wed Jun 24 21:53:45 PDT 2009,NO_QUERY,Jennylovesyou6,@Carale I'm currently watching Season 4 Episod...,"[season, episode, wa, killing, eli, carale, du..."
19998,0,1691228340,Sun May 03 17:23:21 PDT 2009,NO_QUERY,Ciwwaffan10111,You know its bad when you know what episode of...,"[house, holy, social, episode, show, shit, life]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [82]:
list_a = []
#for every row (x), got trough every word (i)
for x in sample["text_clean"]:
    for i in x:
        list_a.append(i)


In [83]:
list_b = nltk.FreqDist(list_a)

words_5 = list(list_b.keys())[:5000]
words_5


['txt',
 'wal',
 'amp',
 'shop',
 'stop',
 'mall',
 'mart',
 'cleaning',
 'bored',
 'cv',
 'chalk',
 'bought',
 'stuff',
 'sidewalk',
 'back',
 'bed',
 'sleep',
 'cool',
 'hr',
 'lay',
 'friend',
 'wrote',
 'poem',
 'nice',
 'itz',
 'feeling',
 'answer',
 'wanted',
 'suck',
 'fun',
 'sitting',
 'anymore',
 'wan',
 'sox',
 'work',
 'instead',
 'white',
 'reaaaaad',
 'watch',
 'tease',
 'kristinburns',
 'text',
 'sooo',
 'twitter',
 'worry',
 'vega',
 'mariahcarey',
 'jazz',
 'feel',
 'wa',
 'quot',
 'missing',
 'sleepting',
 'ntts',
 'confused',
 'sad',
 'xo',
 'star',
 'twittervers',
 'week',
 'night',
 'make',
 'story',
 'unfourtantly',
 'funny',
 'boner',
 'guess',
 'clubpenguinfan',
 'moment',
 'day',
 'mandyyjirouxx',
 'mandy',
 'hope',
 'goodmorning',
 'mum',
 'dad',
 'watching',
 'sore',
 'throat',
 'believe',
 'raining',
 'lot',
 'guy',
 'r',
 'reason',
 'lesliesinks',
 'bunch',
 'tied',
 'king',
 'yourmotherr',
 'love',
 'october',
 'gilmore',
 'sky',
 'lion',
 'happy',
 'hate'

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [85]:
sample

,target,id,date,flag,user,text,text_clean
0,0,2262449609,Sat Jun 20 22:32:03 PDT 2009,NO_QUERY,thorgalsen,"cvs, mall, stop &amp; shop, wal-mart, &amp; cl...","[txt, wal, amp, shop, stop, mall, mart, cleani..."
1,4,2185030529,Mon Jun 15 16:46:50 PDT 2009,NO_QUERY,haliiii,Just bought so much stuff from cvs... 3D SIDEW...,"[chalk, bought, stuff, sidewalk, cv]"
2,4,2016499230,Wed Jun 03 06:53:03 PDT 2009,NO_QUERY,JRink22,4 hrs of sleep - not cool. back to bed ... MY ...,"[back, bed, sleep, cool, hr]"
3,4,1956836172,Thu May 28 22:48:52 PDT 2009,NO_QUERY,MzCandyLand,Itz always nice having a friend who is willing...,"[lay, friend, wrote, poem, nice, itz, feeling]"
4,0,1752440879,Sat May 09 21:55:59 PDT 2009,NO_QUERY,SammyJonas,didn't get the answer she wanted... This reall...,"[answer, wanted, suck]"
...,...,...,...,...,...,...,...
19995,4,1977985185,Sat May 30 22:09:37 PDT 2009,NO_QUERY,aka55,@CateP36 slightly more domesticated than most.,"[slightly, domesticated, catep]"
19996,0,2047804404,Fri Jun 05 14:12:22 PDT 2009,NO_QUERY,rlangdon,@SarahPrevette Oh my gosh! Sarah! Please be ok...,"[long, cent, live, sarah, sarahprevette, gosh]"
19997,0,2322001831,Wed Jun 24 21:53:45 PDT 2009,NO_QUERY,Jennylovesyou6,@Carale I'm currently watching Season 4 Episod...,"[season, episode, wa, killing, eli, carale, du..."
19998,0,1691228340,Sun May 03 17:23:21 PDT 2009,NO_QUERY,Ciwwaffan10111,You know its bad when you know what episode of...,"[house, holy, social, episode, show, shit, life]"


In [86]:
def find_features(doc):
    words = set(doc)
    features = {}
    for i in words_5:
        features[i] = (i in words)
    return features    

In [87]:
features_set = [(find_features(tweet), target) for (tweet, target) in list(zip(sample['text_clean'],sample['target']))]


In [88]:
print(len(features_set))

20000


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [90]:
train_set, test_set = features_set[:3000], features_set[3000:]

In [91]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [92]:
classifier.show_most_informative_features(15)

Most Informative Features
                    sick = True                0 : 4      =     19.4 : 1.0
                    hurt = True                0 : 4      =     14.8 : 1.0
                    hate = True                0 : 4      =     13.6 : 1.0
                     sad = True                0 : 4      =     11.9 : 1.0
                   enjoy = True                4 : 0      =     11.2 : 1.0
                 anymore = True                0 : 4      =      8.9 : 1.0
               listening = True                4 : 0      =      7.8 : 1.0
                    sore = True                0 : 4      =      7.5 : 1.0
                    blog = True                4 : 0      =      7.1 : 1.0
                 amazing = True                4 : 0      =      6.5 : 1.0
                    gift = True                4 : 0      =      6.4 : 1.0
                    rock = True                4 : 0      =      6.4 : 1.0
                   smile = True                4 : 0      =      6.4 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [93]:
print("Naive Bayes Algo Accuracy percent:", (nltk.classify.accuracy(classifier, test_set))*100)

Naive Bayes Algo Accuracy percent: 66.5235294117647


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here

In [ ]:
!git add .

In [ ]:
!git commit -m'finished'

In [ ]:
!git push